In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import datetime
import iisignature as sig
from tqdm import tqdm
from plotly.offline import init_notebook_mode, iplot
from plot_metric.functions import BinaryClassification
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.svm import TimeSeriesSVC
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline


init_notebook_mode(connected=True) ## plotly init
n_splits = 3

In [17]:
#data import
filenames_list = os.listdir('SMNI_CMI_TRAIN/Train') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('SMNI_CMI_TRAIN/Train/' + file_name) ## read from the file to df
    EEG_data = EEG_data.append(temp_df) ## add the file data to the main df

EEG_data = EEG_data.drop(['Unnamed: 0'], axis=1) ## remove the unused column
EEG_data.loc[EEG_data['matching condition'] == 'S2 nomatch,', 'matching condition'] =  'S2 nomatch' ## remove comma sign from stimulus name

468


  0%|          | 0/468 [00:00<?, ?it/s]C:\Users\yousi\AppData\Local\Temp\ipykernel_23212\3614737585.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\yousi\AppData\Local\Temp\ipykernel_23212\3614737585.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\yousi\AppData\Local\Temp\ipykernel_23212\3614737585.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\yousi\AppData\Local\Temp\ipykernel_23212\3614737585.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

  1%|          | 4/468 [00:00<00:12, 38.43it/s]C:\Users\yousi\AppData\Local\Temp\ipykernel_23212\3614737585.py:7: FutureWarning:

The frame.append method

In [18]:
#example try for create train set
temp_df = pd.read_csv('SMNI_CMI_TRAIN/Train/Data1.csv')
# print(temp_df['subject identifier'][0])
# print(temp_df["matching condition"][0])
# print(np.array(temp_df["sensor value"]).reshape([64,256]))

In [19]:
#S1: S1 obj - a single object shown;
s1 = 0
#S12: S2 nomatch - object 2 shown in a non matching condition (S1 differed from S2)
s12 = 0
#S21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
s21 = 0
s1_X_train = np.zeros((160,256,64))
s1_y_train = np.zeros(160)
s21_X_train = np.zeros((159,256,64))
s21_y_train = np.zeros(159)
s12_X_train = np.zeros((149,256,64))
s12_y_train = np.zeros(149)


classifier = {'a':1,'c':0}

filenames_list = os.listdir('SMNI_CMI_TRAIN/Train') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    temp_df = pd.read_csv('SMNI_CMI_TRAIN/Train/' + file_name) ## read from the file to df
    if temp_df["matching condition"][0] == "S1 obj":
        s1_X_train[s1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s1_y_train[s1] = classifier[temp_df['subject identifier'][0]]
        s1 += 1
    if temp_df["matching condition"][0] == "S2 match":
        s21_X_train[s21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s21_y_train[s21] = classifier[temp_df['subject identifier'][0]]
        s21 += 1
    if temp_df["matching condition"][0] == "S2 nomatch,":
        s12_X_train[s12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
        s12_y_train[s12] = classifier[temp_df['subject identifier'][0]]
        s12 += 1
print(s1)
print(s21)
print(s12)

468


100%|██████████| 468/468 [00:04<00:00, 94.20it/s]

160
159
149


In [20]:
#t1: S1 obj - a single object shown;
t1 = 0
#t12: S2 nomatch - object 2 shown in a non matching condition (S1 differed from S2)
t12 = 0
#t21: S2 match - object 2 shown in a matching condition (S1 was identical to S2),
t21 = 0
t1_X_test = np.zeros((160,256,64))
t1_y_test = np.zeros(160)
t21_X_test = np.zeros((160,256,64))
t21_y_test = np.zeros(160)
t12_X_test = np.zeros((160,256,64))
t12_y_test = np.zeros(160)


classifier = {'a':1,'c':0}

filenames_list = os.listdir('SMNI_CMI_TEST') ## list of file names in the directory
EEG_data = pd.DataFrame({}) ## create an empty df that will hold data from each file
print(len(filenames_list))
for file_name in tqdm(filenames_list):
    if file_name == "Test":
        pass
    else:
        temp_df = pd.read_csv('SMNI_CMI_TEST/' + file_name) ## read from the file to df
        if temp_df["matching condition"][0] == "S1 obj":
            t1_X_test[t1] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t1_y_test[t1] = classifier[temp_df['subject identifier'][0]]
            t1 += 1
        if temp_df["matching condition"][0] == "S2 match":
            t21_X_test[t21] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t21_y_test[t21] = classifier[temp_df['subject identifier'][0]]
            t21 += 1
        if temp_df["matching condition"][0] == "S2 nomatch,":
            t12_X_test[t12] = np.transpose(np.array(temp_df["sensor value"]).reshape([64,256]))
            t12_y_test[t12] = classifier[temp_df['subject identifier'][0]]
            t12 += 1
print(t1)
print(t21)
print(t12)

481


100%|██████████| 481/481 [00:05<00:00, 94.62it/s]

160
160
160


In [22]:
pipeline = GridSearchCV(
    Pipeline([
            ('normalize', TimeSeriesScalerMinMax()),
            ('knn', KNeighborsTimeSeriesClassifier())
    ]),
    {'knn__n_neighbors': [3, 20], 'knn__weights': ['uniform', 'distance']},
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0),
    n_jobs=-1
)
pipeline.fit(s1_X_train, s1_y_train)
print(pipeline.score(t1_X_test, t1_y_test))

0.78125


In [ ]:
clf = TimeSeriesSVC(C=1.0, kernel="gak", n_jobs=10)
clf.fit(s1_X_train, s1_y_train)
print(clf.score(t1_X_test, t1_y_test))

In [ ]:
s1_sig_train = sig.sig(s1_X_train, 2)
s1_sig_test = sig.sig(t1_X_test, 2)

In [ ]:
clf = LogisticRegression()
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Logistic Regression")
plt.show()
score = roc_auc_score(t1_y_test, y_pred_proba)
print("score is " + str(score))

In [ ]:
clf = SVC(probability=True)
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)
bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using SVC")
plt.show()
score = roc_auc_score(t1_y_test, y_pred_proba)
print("score is " + str(score))

In [ ]:
for i in tqdm(range(3,20)):
    knn = KNeighborsClassifier(n_neighbors=i, n_jobs=10)
    knn.fit(s1_sig_train, s1_y_train)
    y_pred = knn.predict(s1_sig_test)
    print(accuracy_score(t1_y_test, y_pred))

In [ ]:
clf = AdaBoostClassifier()
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using AdaBoostClassifier")
plt.show()
score = roc_auc_score(t1_y_test, y_pred_proba)
print("score is " + str(score))

In [ ]:
from tqdm import tqdm
for i in tqdm(range(3,20)):
    knn = KNeighborsTimeSeriesClassifier(n_neighbors=i, n_jobs=10)
    knn.fit(s12_X_train, s12_y_train)
    print(knn.score(t12_X_test,t12_y_test))

In [ ]:
start = datetime.datetime.now()
clf = TimeSeriesSVC(C=1.0, kernel="gak")
clf.fit(s12_X_train, s12_y_train)
score = clf.score(t12_X_test, t12_y_test)
end = datetime.datetime.now()
print(score)
print(end-start)

In [ ]:
s12_sig_train = sig.sig(s12_X_train, 2)
s12_sig_test = sig.sig(t12_X_test, 2)

In [ ]:
start = datetime.datetime.now()
rf = RandomForestClassifier(random_state=0)
parameters = param_grid = {'min_weight_fraction_leaf': [0.01, 0.1, 0.5],
                           'bootstrap': [True, False],
                           'max_depth': (2, 5, 10),
                           'max_leaf_nodes': (2, 5, 10),
                           'n_estimators': (100, 200)}
clf = GridSearchCV(rf, parameters, n_jobs=-1)
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Random Forest")
plt.show()
score = roc_auc_score(t12_y_test, y_pred_proba)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
rf = RandomForestClassifier(random_state=0)
parameters = param_grid = {'min_weight_fraction_leaf': [0.01, 0.1, 0.5],
                           'bootstrap': [True, False],
                           'max_depth': (2, 5, 10),
                           'max_leaf_nodes': (2, 5, 10),
                           'n_estimators': (100, 200)}
clf = GridSearchCV(rf, parameters, n_jobs=-1)
clf.fit(s1_sig_train, s1_y_train)
y_pred_proba = clf.predict_proba(s1_sig_test)[:,1]
y_pred = clf.predict(s1_sig_test)

bc = BinaryClassification(t1_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using Random Forest")
plt.show()
score = roc_auc_score(t1_y_test, y_pred_proba)
print("score is " + str(score))

In [ ]:
start = datetime.datetime.now()
clf = LogisticRegression()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)


bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using LogisticRegression")
plt.show()
score = roc_auc_score(t12_y_test, y_pred_proba)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
start = datetime.datetime.now()
clf = SVC(probability = True)
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)


bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using SVC")
plt.show()
score = roc_auc_score(t12_y_test, y_pred_proba)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
for i in tqdm(range(3,20)):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(s12_sig_train, s12_y_train)
    y_pred = knn.predict(s12_sig_test)
    print(accuracy_score(t12_y_test, y_pred))

In [ ]:
start = datetime.datetime.now()
clf = AdaBoostClassifier()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)


bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using AdaBoostClassifier")
plt.show()
score = roc_auc_score(t12_y_test, y_pred_proba)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)

In [ ]:
start = datetime.datetime.now()
clf = GradientBoostingClassifier()
clf.fit(s12_sig_train, s12_y_train)
y_pred_proba = clf.predict_proba(s12_sig_test)[:,1]
y_pred = clf.predict(s12_sig_test)

bc = BinaryClassification(t12_y_test, y_pred_proba, labels=["Class 0", "Class 1"])
# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.title("Receiver Operating Characteristic Using AdaBoostClassifier")
plt.show()
score = roc_auc_score(t12_y_test, y_pred_proba)
end = datetime.datetime.now()
print("score is " + str(score))
print(end-start)